In [ ]:
# Deep Insights (Geometry Fixed - Centroid)
# "Bentech Intelligence Unit" - Deep Insight Module
# ---------------------------------------------------------
#'ST_CENTROID' kullanarak kareleri NOKTA'ya (Point) çeviriyoruz.
# Ayrıca ham Polygon string'ini 'ST_GEOGFROMTEXT' ile gerçek coğrafi veriye dönüştürüyoruz.
# ---------------------------------------------------------

from google.colab import auth
from google.cloud import bigquery
import pandas as pd

# 1. Kimlik Doğrulama
auth.authenticate_user()
print("✅ Kimlik doğrulama başarılı!")

PROJECT_ID = 'YOUR_PROJECT_ID'
DATASET_ID = 'milano_mobile_2013'

client = bigquery.Client(project=PROJECT_ID)
print(f"🔗 BigQuery Bağlantısı Kuruldu: {PROJECT_ID}.{DATASET_ID}")

def create_deep_insight_tables_fixed_geo():
    print("\n🌍 DEEP INSIGHTS: GEOMETRİ DÜZELTİLİYOR (POLYGON -> POINT)...")
    print("="*60)

    # ---------------------------------------------------------
    # 1. DIJITAL SOYLULAŞTIRMA (DIGITAL GENTRIFICATION)
    # Düzeltme: ST_CENTROID(ST_GEOGFROMTEXT(g.geometry))
    # ---------------------------------------------------------
    query_dgi = f"""
    CREATE OR REPLACE TABLE `{PROJECT_ID}.{DATASET_ID}.analysis_digital_gentrification` AS
    WITH cell_stats AS (
        SELECT
            t.CellID,
            SUM(t.internet_winsorized) as total_internet,
            SUM(t.smsin_winsorized + t.smsout_clean) as total_sms
        FROM `{PROJECT_ID}.{DATASET_ID}.traffic_winsorized` t
        GROUP BY 1
    )
    SELECT
        c.CellID,

        -- KRİTİK DÜZELTME: Polygon'u Noktaya Çevir (Bubble Map için şart)
        ST_CENTROID(ST_GEOGFROMTEXT(g.geometry)) as geometry,

        c.total_internet,
        c.total_sms,
        ROUND(SAFE_DIVIDE(c.total_internet, (c.total_sms + 1)), 2) as dgi_score,
        CASE
            WHEN SAFE_DIVIDE(c.total_internet, (c.total_sms + 1)) > 3.0 THEN 'High-End (Modern)'
            WHEN SAFE_DIVIDE(c.total_internet, (c.total_sms + 1)) BETWEEN 1.0 AND 3.0 THEN 'Mid-Market'
            ELSE 'Traditional / Budget'
        END as socio_economic_segment

    FROM cell_stats c
    LEFT JOIN `{PROJECT_ID}.{DATASET_ID}.grid_locations` g
        ON c.CellID = g.CellID
    WHERE c.total_internet > 100
    """

    # ---------------------------------------------------------
    # 2. OLAY SONRASI DAĞILIM (RETAIL OPPORTUNITY)
    # Düzeltme: ST_CENTROID(ST_GEOGFROMTEXT(g.geometry))
    # ---------------------------------------------------------
    query_flow = f"""
    CREATE OR REPLACE TABLE `{PROJECT_ID}.{DATASET_ID}.analysis_retail_opportunity` AS
    SELECT
        t.CellID,

        -- KRİTİK DÜZELTME: Polygon'u Noktaya Çevir (Heatmap için şart)
        ANY_VALUE(ST_CENTROID(ST_GEOGFROMTEXT(g.geometry))) as geometry,

        EXTRACT(HOUR FROM t.datetime) as hour,
        SUM(t.internet_winsorized + t.callin_clean + t.callout_clean) as traffic_volume,
        CASE
            WHEN c.Cluster = 2 THEN 'Target Zone (Navigli/Bars)'
            WHEN c.Cluster = 1 THEN 'Dead Zone (Office)'
            ELSE 'Transit / Residential'
        END as zone_type

    FROM `{PROJECT_ID}.{DATASET_ID}.traffic_winsorized` t
    LEFT JOIN `{PROJECT_ID}.{DATASET_ID}.grid_locations` g ON t.CellID = g.CellID
    LEFT JOIN `{PROJECT_ID}.{DATASET_ID}.analysis_land_use_clusters` c ON t.CellID = c.CellID
    WHERE
        DATE(t.datetime) = '2013-11-03'
        AND EXTRACT(HOUR FROM t.datetime) BETWEEN 20 AND 23
    GROUP BY 1, 3, 5
    """

    try:
        print("⏳ Tablo 1 Düzeltiliyor (Gentrification - Points)...")
        client.query(query_dgi).result()
        print(f"✅ Tablo 1 Hazır: analysis_digital_gentrification")

        print("⏳ Tablo 2 Düzeltiliyor (Retail Opportunity - Points)...")
        client.query(query_flow).result()
        print(f"✅ Tablo 2 Hazır: analysis_retail_opportunity")
        print("\n🚀 GEOMETRİLER DÜZELDİ! Artık Looker Studio'da 'Baloncuk' ve 'Isı' haritaları çalışacak.")

    except Exception as e:
        print(f"\n❌ HATA: {e}")

create_deep_insight_tables_fixed_geo()